In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
patches = torch.randn(10, 32, 32, 4)

In [3]:
torch.arange(32), torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [9]:
y, x = torch.meshgrid(torch.arange(5), torch.arange(4), indexing='ij')

In [10]:
y, x

(tensor([[0, 0, 0, 0],
         [1, 1, 1, 1],
         [2, 2, 2, 2],
         [3, 3, 3, 3],
         [4, 4, 4, 4]]),
 tensor([[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]]))

In [153]:
y, x = torch.meshgrid(torch.arange(5), torch.arange(4), indexing='xy')
y.v, x.v

(tensor[4, 5] i64 n=20 x∈[0, 4] μ=2.000 σ=1.451
 tensor([[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]),
 tensor[4, 5] i64 n=20 x∈[0, 3] μ=1.500 σ=1.147
 tensor([[0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3]]))

In [12]:
omega = torch.arange(128 //4) / (128 // 4 -1)  # linsapce같은 역할

In [18]:
omega = 1 / (10000 ** omega) # 점점 작아지도록 [dim//4]

In [23]:
y.flatten()[:None].shape

torch.Size([20])

In [24]:
# [20, 1] * [1, 32] => [20, 32]
y = y.flatten()[:,None] * omega[None, :]

In [29]:
x = x.flatten()[:, None] * omega[None, :] 

In [30]:
pe = torch.cat((x.sin(), x.cos(), y.sin(), y.cos()), dim=1)

In [31]:

pe.shape

torch.Size([20, 128])

In [35]:
import lovely_tensors as lt

In [36]:
lt.monkey_patch()

print(pe)

tensor[20, 128] n=2560 x∈[-0.990, 1.000] μ=0.513 σ=0.487


In [39]:
print(pe.deeper(100))

tensor[20, 128] n=2560 x∈[-0.990, 1.000] μ=0.513 σ=0.487
  tensor[128] x∈[0., 1.000] μ=0.500 σ=0.502
  tensor[128] x∈[0., 1.000] μ=0.520 σ=0.481
  tensor[128] x∈[-0.416, 1.000] μ=0.517 σ=0.484
  tensor[128] x∈[-0.990, 1.000] μ=0.500 σ=0.502
  tensor[128] x∈[-0.986, 1.000] μ=0.482 σ=0.520
  tensor[128] x∈[0., 1.000] μ=0.520 σ=0.481
  tensor[128] x∈[1.000e-04, 1.000] μ=0.540 σ=0.458
  tensor[128] x∈[-0.416, 1.000] μ=0.537 σ=0.461
  tensor[128] x∈[-0.990, 1.000] μ=0.520 σ=0.481
  ...


In [62]:
torch.randn(10, 4).chunk(2, dim=-1)

(tensor[10, 2] n=20 x∈[-1.664, 2.294] μ=0.309 σ=0.992,
 tensor[10, 2] n=20 x∈[-1.542, 2.610] μ=0.201 σ=1.083)

In [64]:
torch.nn.functional.gelu(torch.randn(10, 2)) * torch.randn(10, 2)

tensor[10, 2] n=20 x∈[-2.300, 0.269] μ=-0.337 σ=0.770

In [59]:
torch.nn.functional.layer_norm(x, x.shape[-1:], torch.ones(32), torch.zeros(32))a

tensor[1, 10, 32] n=320 x∈[-3.325, 2.751] μ=2.235e-09 σ=1.002

In [60]:
torch.nn.functional.layer_norm(x, x.shape[-1:])

tensor[1, 10, 32] n=320 x∈[-3.325, 2.751] μ=2.235e-09 σ=1.002

In [65]:
class GEGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return torch.nn.functional.gelu(gate) * x

In [74]:
class LayerNorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(dim))
        self.register_buffer('beta', torch.zeros(dim))

    def forward(self, x):
        return F.layer_norm(x, x.shape[-1:], self.gamma, self.beta)

In [77]:
nn.Sequential(LayerNorm(10),
              nn.Linear(10, 20),
              GEGLU(),
              nn.Linear(10, 10)
              )(torch.randn(3, 33, 44, 10))

tensor[3, 33, 44, 10] n=43560 x∈[-0.893, 0.824] μ=-0.097 σ=0.208 grad AddBackward0

In [79]:
int(10*4*2/3)

26

In [80]:
from torchaudio.transforms import Spectrogram, TimeStretch, FrequencyMasking, TimeMasking

In [81]:
def pair(t):
    return (t, t) if not isinstance(t, tuple) else t

In [83]:
class AudioSpectrogramTransformer(nn.Module):
    def __init__(self, dim, depth, patch_size=16, dim_head=64, heads=8,
                 attn_dropout=0., ff_mult=4, ff_dropout=0.,spec_n_fft=128):
        

([1, 2, 3, 4], [1, 2, 3, 4])

In [84]:
from einops import rearrange, repeat, reduce, unpack, pack
from einops.layers.torch import Rearrange

In [89]:
patch_size = pair(16)
patch_input_dim = patch_size[0] * patch_size[1]

In [105]:
nn.Sequential(Rearrange('b (h p1) (w p2) -> b h w (p1 p2)', p1=patch_size[0],p2=patch_size[1]),
              nn.LayerNorm(patch_input_dim),
              nn.Linear(patch_input_dim, 10),
              nn.LayerNorm(10))(torch.randn(10, 2, 32, 32))

EinopsError: Expected 3 dimensions, got 4

In [104]:
torch.randn((10,2,3,4,5))[1:].shape

torch.Size([9, 2, 3, 4, 5])

In [127]:
torch.stack(torch.meshgrid(
            torch.arange(4),
            torch.arange(5)
        , indexing = 'ij'),dim=-1)[2][0].v

tensor[2] i64 μ=1.000 σ=1.414 [2, 0]
tensor([2, 0])

In [134]:
grid = torch.stack(torch.meshgrid(
            torch.arange(16),
            torch.arange(16)
        , indexing = 'ij'),dim=-1)

In [135]:
grid

tensor[16, 16, 2] i64 n=512 x∈[0, 15] μ=7.500 σ=4.614

In [136]:
grid = rearrange(grid, '... c -> (...) c')

In [137]:
grid

tensor[256, 2] i64 n=512 x∈[0, 15] μ=7.500 σ=4.614

In [138]:
def posemb_sincos_2d(patches, temperature = 10000, dtype = torch.float32):
    _, h, w, dim, device, dtype = *patches.shape, patches.device, patches.dtype

    y, x = torch.meshgrid(torch.arange(h, device = device), torch.arange(w, device = device), indexing = 'ij')
    assert (dim % 4) == 0, 'feature dimension must be multiple of 4 for sincos emb'

    omega = torch.arange(dim // 4, device = device) / (dim // 4 - 1)
    omega = 1. / (temperature ** omega)

    y = y.flatten()[:, None] * omega[None, :]
    x = x.flatten()[:, None] * omega[None, :] 

    pe = torch.cat((x.sin(), x.cos(), y.sin(), y.cos()), dim = 1)
    pe = pe.type(dtype)

    return rearrange(pe, '(h w) d -> h w d', h = h, w = w)


In [139]:
x = torch.randn(10, 16, 16, 128)
x = x + posemb_sincos_2d(x)

In [145]:
x

tensor[10, 16, 16, 128] n=327680 x∈[-4.946, 5.683] μ=0.452 σ=1.138

In [ ]:
rearrange(x, 'b ... c -> b (...) c')[0].chans(cmap='coolwarm', scale=4)

In [151]:
x = torch.randn(7, 3, 4)

rearrange(x, '... i c -> ... i 1 c')

tensor[7, 3, 1, 4] n=84 x∈[-3.091, 2.171] μ=0.123 σ=1.050